In [ ]:
## Import statements

import pandas as pd
import numpy as np

In [2]:
## Load file and extract sheets and values

# Load file 
xl = pd.ExcelFile('HighCr_input.xlsx')

# Load sheet as numpy array
HD = xl.parse('HessDeepAvg').as_matrix()
Gk = xl.parse('Gakkel').as_matrix()
SW = xl.parse('SWIR').as_matrix()
BW = xl.parse('BryndziaWood').as_matrix()
SW78 = xl.parse('SWIR_Plag').as_matrix()
Tonga = xl.parse('Tonga').as_matrix()

# Dictionary of values
D = {'Cr# (spl)':0,
     'Cr_std':1,
     'Mg# (spl)':2,
     'Mg_std':3,
     'Fe$^{3+}$/$\Sigma$Fe (spl)':4,
     'Fe3+/tFe_err':6,
     'wt% TiO$_2$ (spl)':7,
     'TiO2_std':8,
     'wt% NiO (spl)':9,
     'Fo# (olv)':10,
     '$X_{Fe}^{M1}*X_{Fe}^{M2}$ (opx)':11,
     'T (°C)':12,
     '$log(f_{O2})$, $\Delta$QFM':13,
     'plag':14,
     'ol/opx':15,
     'Al2O3':16,
     'fO2_err_pos':17,
     'fO2_err_neg':18,
     'T_std':19,
     'Ni_std':20,
     'Fo_prec':21,
     'XM1XM2_prec':22,
     'trace':23,
     'cpx_trace':24,
     'opx_trace':66
    }

# Dictionary of error bar value
D_err = {'Cr# (spl)':[1,1],
        'Mg# (spl)':[3,3],
        'Fe$^{3+}$/$\Sigma$Fe (spl)':[6,6],
        'wt% TiO$_2$ (spl)':[8,8],
        '$log(f_{O2})$, $\Delta$QFM': [17,18],
        'T (°C)':[19,19],
        'wt% NiO (spl)':[20,20],
        'Fo# (olv)':[21,21],
        '$X_{Fe}^{M1}*X_{Fe}^{M2}$ (opx)':[22,22]}

Global = xl.parse('Global').as_matrix()
Cr_global = Global[:,0]
Yb_global = Global[:,1]
TiO2_global = Global[:,2]

/Applications/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
/Applications/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if __name__ == '__main__':
/Applications/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # Remove the CWD from sys.path while we load stuff.
/Applications/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # This is added back by InteractiveShellApp.init_path()
/Applications/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] =

In [3]:
## Trace element set-up

def trace_setup(sheet_name, data):
    cols = list(xl.parse(sheet_name).columns.values)

    cpx_tcol0 = cols.index('Li')
    cpx_tcol1 = cols.index('U')+1

    opx_tcol0 = cols.index('Li.1')
    opx_tcol1 = cols.index('U.1')+1

    trace_names = cols[cpx_tcol0:cpx_tcol1]

    cpx_trace_dicts = []
    for i in range(len(data)):
        trace_dict = {}
        for j, col in enumerate(trace_names):
            trace_dict[col] = data[i,cpx_tcol0 + j]
        trace_dict['Ti'] = trace_dict['TiO2%']*10000*47.867/79.866
        trace_dict['Gd'] = trace_dict['Gd(Dy)']
        cpx_trace_dicts.append(trace_dict)

    opx_trace_dicts = []
    for i in range(len(data)):
        trace_dict = {}
        for j, col in enumerate(trace_names):
            trace_dict[col] = data[i,opx_tcol0 + j]
        trace_dict['Ti'] = trace_dict['TiO2%']*10000*47.867/79.866 #ppm Ti from wt% TiO2
        trace_dict['Gd'] = trace_dict['Gd(Dy)']
        opx_trace_dicts.append(trace_dict)
    
    return cpx_trace_dicts, opx_trace_dicts
    

# Setting up normalizing values
Trace_info = xl.parse('Trace_info').as_matrix()
cols = list(xl.parse('Trace_info').columns.values)

cpx_tcol0 = cols.index('Li')
cpx_tcol1 = cols.index('U')+1

#opx_tcol0 = cols.index('Li.1')
#opx_tcol1 = cols.index('U.1')+1

trace_names = cols[cpx_tcol0:cpx_tcol1]

chon = {}
for j, col in enumerate(trace_names):
    chon[col] = Trace_info[0,j]
chon['Gd'] = chon['Gd(Dy)']
    
PUM = {}
for j, col in enumerate(trace_names):
    PUM[col] = Trace_info[1,j]
PUM['Ti'] = 1265 # ppm
PUM['Gd'] = PUM['Gd(Dy)']

cpx_trace_dicts, opx_trace_dicts = trace_setup('SWIR', SW)
cpx_trace_dicts_SWplag, opx_trace_dicts_SWplag = trace_setup('SWIR_Plag', SW78)

/Applications/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:36: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
